<a href="https://colab.research.google.com/github/simone-viozzi/drive-transfere-ownership/blob/master/transfere%20ownership.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# intro
Currently google drive don't have a function/api to change owner to files and transfere the quota from a user to another.
So i creaded this google colab that use [the drive package](https://github.com/odeke-em/drive/) to transfere the ownership of folder and build-in file types (like google docs and spreadsheet) and dowload from an account and upload from another for all the other files (the one that actually take space in drive).

I did this in colab because it's bandwith is pretty high and does't consume local resources.

So the roadmap is:


1.   install the packages 
2.   initialize the first drive (the one from we will remove the files)
3.   download the folder that we intend to transfere
4.   to speed up the process i writed some python code to determine which file types are build-in (so we can transfere ownership) and which file are external (so we must dowload and reupload them)
5.    now we can transfere the ownership of folder and build-in types (tested with google docs and spreadsheet, it SHOULD work even if you open a txt in drive and it become a build-in type)
6.   we copy the desidered folder to another folder to keep a copy
7.   and than we can trash all the files that are not build-in (pdf, zip, etc) this step actually trash an entire folder recursively so we need to be very carefully. i DIDN'T test to recover the files from the trash and see what happen, so maybe the tree of folder will be lost. so very carefull!
8.   we finished for the first account so we can init the second account
9.   we move to the folder that we transfered (if the second account had the folder share before we need to navigate to that folder)
10.  and dowload it to create the tree of folder and all the build-in files
11.  last step we copy the folder and upload it!

If in this process the machine terminate in mid operation we will have a total mess to use it carefully!

This is a total alpha and need to be perfected!


# first step
we install some useful packages, go and drive

In [0]:
!sudo apt-get install tree
!sudo apt-get install file
!wget https://dl.google.com/go/go1.14.2.linux-amd64.tar.gz
!tar -C /usr/local -xzf go1.14.2.linux-amd64.tar.gz
import os
os.environ['PATH'] += ":/usr/local/go/bin"
!go get -u github.com/odeke-em/drive/cmd/drive
os.environ['PATH'] += ":/root/go/bin"

# second step
we initialize the first account and pull some directories to build the tree of folders and navigate locally to the desidered folder

In [0]:
!mkdir first_account
%cd first_account
!drive init
!drive index -no-prompt -quiet
!drive pull -directories -depth 4 -quiet -no-prompt -fix-clashes

just to assure that evriting is ok

In [0]:
!pwd
!tree -shC -L 2

# step three
we must navigate to the folder that we intend to transfere

In [0]:
path="/content/first_account/folder1"
folder_to_transfere="folder2"

this is to assure that the path is correct, if not you need to modify the path above and than re-run the cell below.

In [0]:
!echo $path
%cd $path
%cd $folder_to_transfere
!pwd

and than we dowload the desidered folder, google have fast internet but it will take time depending from the size of the files. drive have an issue with colab so will show nothing (the progress bar make colab crash).

In [0]:
!drive pull -directories -quiet -no-prompt -fix-clashes
!drive pull -quiet -no-prompt -fix-clashes

just to assure that everithing is ok

In [0]:
!drive stat -recursive -depth 2

# step four
to speed up the process of changing owner i try so select only the files whose ownership can be transferred. I DIDN'T test it enough to say that it work in any case so proceed with caution.

In [0]:
from collections import Counter

google_files = !find . -type f -name '*.desktop'

protected_file_types = []
for f in google_files:
  f = f.split('/')[-1]
  if f.count('.') == 2:
    protected_file_types.append(f.split('.')[-2])
protected_file_types = list(dict.fromkeys(protected_file_types))
protected_file_types.append('desktop')
print(protected_file_types)

file_types = !find . -type f -name '*.*' | sed 's|.*\.||' | sort -u

file_types = list((Counter(file_types) - Counter(protected_file_types)).elements())
print(file_types)

with open('/content/protected_file_types.txt', 'w') as f:
    for ft in protected_file_types:
      f.write(ft + '\n')
with open('/content/file_types.txt', 'w') as f:
    for ft in file_types:
      f.write(ft + '\n')

# step five
we can transfere the ownership basing on what we found above. You need to write the `second_email` manually, doublecheck it because this can't be undone!



In [0]:
%%bash
filename=/content/protected_file_types.txt

second_email=jhon@gmail.com

while IFS= read -r f;
do
  find ./ -name "*.$f" -exec drive share -no-prompt -quiet -emails $second_email -role owner {} ';' ;
done < $filename

find ./ -not -name "*.*" -exec drive share -no-prompt -quiet -emails $second_email -role owner {} ';'

to assure that the files actually changed owner

In [0]:
!drive stat -recursive -depth 2

# step six
we copy the desidered folder to a path so we know where it is

In [0]:
%%bash
cd "$path"
pwd
mkdir /content/file\ to\ transfere
cp -r "./../$folder_to_transfere/*" /content/file\ to\ transfere/
cd "./$folder_to_transfere"
pwd

# step seven
we trash all the files whose ownership can't be transferred. doublecheck the find command to assure it find only the desidered files!

In [0]:
%%bash

filename=/content/file_types.txt

while IFS= read -r f;
do
  find ./ -name "*.$f" -exec drive trash -quiet {} ';' ;
done < $filename

# step eight
now we ended for the first account and we can proceed to initialize the second account.

In [0]:
%cd /content/
!mkdir second_account
%cd second_account
!drive init
!drive index -no-prompt -quiet
!drive pull -directories -depth 4 -quiet -no-prompt -fix-clashes

# step nine
we must move to the folder that we are tring to transfere. doublecheck the path and the tree of folder!

In [0]:
%cd "./$folder_to_transfere"
!pwd
!tree -shaC

# step ten 
to avoid errors we must dowload the folder (it will be basically void because we removed all not buld-in files).

In [0]:
!drive pull -quiet -no-prompt -fix-clashes

# step eleven
finally we can copy the folder and upload it!

In [0]:
!cp -r /content/file\ to\ transfere/* ./
!drive push -quiet -no-prompt -fix-clashes

# conclusion
this colab need some work but is a prof of concept that we can use the google bandwith to transfere files between drive accounts. 